In [4]:
import sys
sys.path.append('/Users/marcelbreithaupt/Library/Mobile Documents/com~apple~CloudDocs/ENV/Master/Semester 2/DevOps/Project')

In [12]:
from server.py.game import Game
from server.py.player import Player
from typing import List, Optional, ClassVar
from pydantic import BaseModel
from enum import Enum
import random


class Card(BaseModel):
    suit: str  # card suit (color)
    rank: str  # card rank


class Marble(BaseModel):
    pos: str       # position on board (0 to 95)
    is_save: bool  # true if marble was moved out of kennel and was not yet moved


class PlayerState(BaseModel):
    name: str                  # name of player
    list_card: List[Card]      # list of cards
    list_marble: List[Marble]  # list of marbles


class Action(BaseModel):
    card: Card                 # card to play
    pos_from: Optional[int]    # position to move the marble from
    pos_to: Optional[int]      # position to move the marble to
    card_swap: Optional[Card]  # optional card to swap


class GamePhase(str, Enum):
    SETUP = 'setup'            # before the game has started
    RUNNING = 'running'        # while the game is running
    FINISHED = 'finished'      # when the game is finished


class GameState(BaseModel):
    # Define these as placeholders, and initialize them properly in a class method.
    LIST_SUIT: ClassVar[List[str]] = []
    LIST_RANK: ClassVar[List[str]] = []
    LIST_CARD: ClassVar[List[Card]] = []

    @classmethod
    def initialize_constants(cls):
        """Initialize the class variables."""
        cls.LIST_SUIT = ['♠', '♥', '♦', '♣']
        cls.LIST_RANK = [
            '2', '3', '4', '5', '6', '7', '8', '9', '10',
            'J', 'Q', 'K', 'A', 'JKR'
        ]
        cls.LIST_CARD = [
            Card(suit=suit, rank=rank)
            for suit in cls.LIST_SUIT for rank in cls.LIST_RANK if rank != 'JKR'
        ] + [Card(suit='', rank='JKR')] * 3  # Adding Jokers

    cnt_player: int = 4
    phase: GamePhase
    cnt_round: int
    bool_game_finished: bool
    bool_card_exchanged: bool
    idx_player_started: int
    idx_player_active: int
    list_player: List[PlayerState]
    list_id_card_draw: List[Card]
    list_id_card_discard: List[Card]
    card_active: Optional[Card]


class Dog(Game):
    def __init__(self) -> None:
        self.state = GameState(
            cnt_player=4,
            phase=GamePhase.SETUP,
            cnt_round=0,
            bool_game_finished=False,
            bool_card_exchanged=False,
            idx_player_started=0,
            idx_player_active=0,
            list_player=[
                PlayerState(name=f"Player {i + 1}", list_card=[], list_marble=[]) for i in range(4)
            ],
            list_id_card_draw=GameState.LIST_CARD.copy(),
            list_id_card_discard=[],
            card_active=None
        )
        random.shuffle(self.state.list_id_card_draw)

    def set_state(self, state: GameState) -> None:
        self.state = state

    def get_state(self) -> GameState:
        return self.state

    def print_state(self) -> None:
        print(f"Round: {self.state.cnt_round}, Phase: {self.state.phase}")
        for idx, player in enumerate(self.state.list_player):
            print(f"Player {idx + 1}: Cards - {player.list_card}, Marbles - {player.list_marble}")

    def get_list_action(self) -> List[Action]:
        active_player = self.state.list_player[self.state.idx_player_active]
        actions = []
        for card in active_player.list_card:
            for marble in active_player.list_marble:
                if marble.pos.isdigit() and 0 <= int(marble.pos) < 96:
                    actions.append(Action(card=card, pos_from=int(marble.pos), pos_to=int(marble.pos) + 1))
        return actions

    def apply_action(self, action: Action) -> None:
        if action and action.pos_from is not None and action.pos_to is not None:
            for marble in self.state.list_player[self.state.idx_player_active].list_marble:
                if int(marble.pos) == action.pos_from:
                    marble.pos = str(action.pos_to)
            self.state.list_player[self.state.idx_player_active].list_card.remove(action.card)
            self.state.list_id_card_discard.append(action.card)

    def get_player_view(self, idx_player: int) -> GameState:
        masked_state = self.state.copy()
        for idx, player in enumerate(masked_state.list_player):
            if idx != idx_player:
                player.list_card = [Card(suit="?", rank="?")] * len(player.list_card)
        return masked_state


class RandomPlayer(Player):
    def select_action(self, state: GameState, actions: List[Action]) -> Action:
        if len(actions) > 0:
            return random.choice(actions)
        return None


if __name__ == '__main__':
    game = Dog()
    game.print_state()
    actions = game.get_list_action()
    if actions:
        print(f"Possible actions: {actions}")
        game.apply_action(actions[0])
        game.print_state()

Round: 0, Phase: GamePhase.SETUP
Player 1: Cards - [], Marbles - []
Player 2: Cards - [], Marbles - []
Player 3: Cards - [], Marbles - []
Player 4: Cards - [], Marbles - []
